# New bussines location indicator

### Coursera project capstone

This is the course captstone project for the coursera IBM machine learning specialization

### About the project

Having a good location for your business is an important factor for it's prosperity. although the hot spots in a city for a new venue are common knowledge, usually this places have a very high price given that this knowledge is correlated with higher demand. With some datamining and machine learning, some places cam be discovered as good opportunity’s that does not yet received peoples attention.

This project uses python, foursquare api, machine learning and visualization tools to give some insight on where you should open your business in a city.
It works by ranking the places in city by how hot they are for business and them finding what is the optimal place for a new business that does not collide with other similar business by location but also is on a "hot spot" in the city.

The data will on the amount of venues by location will be collected using the foursquare api for a given city. The app/notebook will be created in a generic way so that people using it can change the city name. 
A map of the city based on the amount of venues will be created and also another one for the similar venues as the person is looking to create.

A score for a given location that will be created by adding value for number of venues using mahalanobis distance as a weight and the using the opposite (decreasing score) by having similar venues.
In the and, points will be clustered together to have useful locations as a suggestion for the new business to open.

### Let's start by importing the libraries we will need

In [1]:
import pandas as pd
import numpy as np
import folium, requests, os, time, itertools, pickle
import folium.plugins as plugins
from bs4 import BeautifulSoup
from urllib.request import urlopen
from geopy.geocoders import Nominatim
from IPython.display import Image

## Defining the place of interest and search area

In [2]:
# this will be our center point in the city
lat_city_center, lng_city_center = -22.727482, -47.648811

# now, create a map with this lat and lng info
map_city = folium.Map(location=[lat_city_center, lng_city_center], zoom_start=14)
folium.CircleMarker(
        [lat_city_center, lng_city_center],
        radius=10,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  

In [3]:
# check if it is correct
map_city

In [141]:
# Defining the search grid size and location

In [4]:
number_x_points = 10
number_y_points = 10

lat_city_center, lng_city_center = -22.727482, -47.648811
# the farthest point of interest in the map
x_max, y_max =  -22.713566, -47.659758

# now we create a distance range in lat and lng distance measure
lat_delta = 2*np.abs(x_max - lat_city_center)
lng_delta = 2*np.abs(y_max - lng_city_center)

In [5]:
# create the matrix of points for use in the map and foursquare
lat_range = np.linspace(lat_city_center - lat_delta, lat_city_center + lat_delta, number_x_points)
lng_range = np.linspace(lng_city_center - lng_delta, lng_city_center + lng_delta, number_y_points)

In [6]:
lat_range

array([-22.755314  , -22.74912911, -22.74294422, -22.73675933,
       -22.73057444, -22.72438956, -22.71820467, -22.71201978,
       -22.70583489, -22.69965   ])

In [7]:
for lat in lat_range:
    for lng in lng_range:
        folium.CircleMarker(
            [lat, lng],
            radius=10,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_city)
map_city

whe can see that we have covered most of the city with every point at around 200m from each other

## Requesting the data from the location with foursquare api

In [8]:
# Client ID and cliet secret key shoud never be stored in the notebook or other script,
# so we read it from os enviroment variable. 

CLIENT_ID = 'OHYNR0DTJAZBPYS2YKPXKPVY5YTP1I1HAKEY5S4BZBP0CVBF' 
CLIENT_SECRET = 'SZ0VR5AMVODFXWLRJME52RASHKPXKMBJBZB0OVXLIRLHUSRM' 
VERSION = '20180605' # Foursquare API version

#CLIENT_ID = os.getenv('CLIENT_ID')          # your Foursquare ID
#CLIENT_SECRET = os.getenv('CLIENT_SECRET')  # your Foursquare Secret

Creating some help functions to get and treat the data received

In [9]:
def get_info(lat, lng):
    '''
    This function call foursquare api with given lattitude and longtude and
    returns the api response as 
    '''
    url = f'https://api.foursquare.com/v2/venues/search?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={250}&limit={500}'
    results = requests.get(url).json()
    if results['meta']['code'] == 200:
        return results['response']
    return False

In [10]:
def count_venues(response):
    '''
    This function parse the response received from the get_info() function
    and treat it to filter only the number of venues in the area and retur it's number
    and a list of lat and lng data in a list o lists format
    '''
    try:
        points = []
        if response['venues']:    
            for i in range(len(response['venues'])):
                try:
                    lat = r['venues'][i]['location']['lat']
                    lng = r['venues'][i]['location']['lng']
                    points.append([lat, lng])
                except:
                    pass
            return len(response['venues']), points
    except:
        pass
    return 0, None

In [11]:
def count_similar(response, similar=['Café', 'Cafe', 'Coffe', 'Coffee Shops']):
    '''
    This function parse the response received from the get_info() function
    and count how many of then has similar text (eg: has the type we are looking for)
    and retur it's number
    '''
    total = 0 # we start with 0 matchs  
    try:
        points = []
        for venue in response['venues']:
            for item in venue['categories']:
                if  item['pluralName'] in similar or item['shortName'] in similar or item['name'] in similar:
                    total += 1
                    try:
                        lat = venue['location']['lat']
                        lng = venue['location']['lng']
                        points.append([lat, lng])
                    except:
                        pass
        return total, points
    except:
        return 0, None

In [13]:
# Let's just check the functions created
r = get_info(lat_city_center, lng_city_center)
print(f'number of coffe shops: {count_similar(r)}')
print(f'number of venues in the center of the city: {count_venues(r)}')
r['venues'][0]['location']['lat']

number of coffe shops: (4, [[-22.728410377806544, -47.6496679173587], [-22.728626, -47.649757], [-22.727285, -47.65005], [-22.728888, -47.64981]])
number of venues in the center of the city: (109, [[-22.727424355059128, -47.648921896561056], [-22.727786577073832, -47.64798471403457], [-22.72776026949104, -47.649056911468506], [-22.72706255188629, -47.64871523436572], [-22.725944134849517, -47.64856624584415], [-22.726414440497845, -47.64845609664917], [-22.727632118073448, -47.64925832512503], [-22.728410377806544, -47.6496679173587], [-22.728482685697028, -47.64965696407798], [-22.728068500444433, -47.648949758969955], [-22.727348413893818, -47.64938328485321], [-22.727035823687157, -47.64866717350626], [-22.727088931467915, -47.64951305103854], [-22.728114, -47.649298], [-22.72756544620964, -47.64834997293369], [-22.727695526961867, -47.649003600443336], [-22.727378020145892, -47.64843167592882], [-22.726199694105176, -47.64813850694653], [-22.726807035053387, -47.64853260333309], [-

-22.727424355059128

Now, we will search in all the opoints in the map

In [146]:
# the flag bellow is to avoid excess use of the api, loading the data if it already exists
new_scrap = True


filename = 'foursquare_data.pk'
if new_scrap:
    full_response, points_coffe, points_venue = [], [], []
    for lat in lat_range:
        line = []
        for lng in lng_range:
            r = get_info(lat, lng)
            similar, pt_temp_cofee = count_similar(r)
            venues, pt_temp_venue = count_venues(r)
            if pt_temp_venue:
                points_venue.extend(pt_temp_venue)
            if pt_temp_cofee:
                points_coffe.extend(pt_temp_cofee)
            line.append((similar, venues))
            time.sleep(1)
        full_response.append(line) 
    outfile = open(filename, 'wb')
    pickle.dump([full_response, points_coffe, points_venue], outfile)
    outfile.close()
else:
    infile = open(filename,'rb')
    full_response, points_coffe, points_venue = pickle.load(infile)
    infile.close()

In [147]:
full_response

[[(0, 190),
  (0, 177),
  (0, 186),
  (0, 191),
  (0, 178),
  (0, 131),
  (0, 155),
  (0, 161),
  (0, 113),
  (0, 104)],
 [(0, 190),
  (0, 190),
  (0, 190),
  (0, 160),
  (0, 186),
  (0, 162),
  (0, 112),
  (0, 147),
  (0, 153),
  (0, 135)],
 [(0, 187),
  (0, 162),
  (1, 188),
  (1, 190),
  (1, 115),
  (0, 183),
  (0, 125),
  (0, 85),
  (0, 145),
  (0, 131)],
 [(0, 185),
  (0, 173),
  (1, 153),
  (4, 193),
  (4, 164),
  (2, 116),
  (2, 180),
  (0, 81),
  (0, 99),
  (0, 135)],
 [(2, 141),
  (1, 145),
  (0, 118),
  (0, 181),
  (4, 172),
  (1, 117),
  (0, 82),
  (0, 64),
  (0, 110),
  (0, 77)],
 [(3, 143),
  (1, 160),
  (0, 171),
  (1, 169),
  (4, 188),
  (2, 130),
  (0, 87),
  (0, 62),
  (0, 135),
  (0, 104)],
 [(1, 180),
  (0, 167),
  (1, 132),
  (3, 194),
  (4, 191),
  (0, 132),
  (0, 87),
  (0, 88),
  (0, 127),
  (0, 79)],
 [(1, 185),
  (2, 181),
  (6, 184),
  (4, 194),
  (1, 85),
  (2, 169),
  (0, 128),
  (0, 76),
  (0, 136),
  (0, 72)],
 [(0, 152),
  (0, 139),
  (2, 120),
  (3, 193)

In [94]:
r

{'venues': [{'id': '5065c89ce4b0b68b5d920977',
   'name': 'Restaurante Germann / Esalq',
   'location': {'address': 'Esalq',
    'lat': -22.700024147970947,
    'lng': -47.633118493898856,
    'labeledLatLngs': [{'label': 'display',
      'lat': -22.700024147970947,
      'lng': -47.633118493898856}],
    'distance': 638,
    'cc': 'BR',
    'city': 'Piracicaba',
    'state': 'SP',
    'country': 'Brasil',
    'formattedAddress': ['Esalq', 'Piracicaba, SP', 'Brasil']},
   'categories': [{'id': '4bf58dd8d48988d1c4941735',
     'name': 'Restaurant',
     'pluralName': 'Restaurants',
     'shortName': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1574216059',
   'hasPerk': False},
  {'id': '565f6215498e1d92699b9d4f',
   'name': 'Kalunga',
   'location': {'address': 'Av. Limeira',
    'lat': -22.703212828376003,
    'lng': -47.62517547156879,
    'labeledLatLngs': [{'label':

In [148]:
points_coffe

[[-22.73719337319173, -47.65508368607477],
 [-22.73719337319173, -47.65508368607477],
 [-22.73804653271318, -47.65082995559482],
 [-22.734132, -47.657055],
 [-22.73719337319173, -47.65508368607477],
 [-22.735312782565494, -47.654751871943816],
 [-22.735857954314223, -47.65362804442607],
 [-22.734132, -47.657055],
 [-22.735857954314223, -47.65362804442607],
 [-22.73804653271318, -47.65082995559482],
 [-22.735312782565494, -47.654751871943816],
 [-22.73719337319173, -47.65508368607477],
 [-22.736636, -47.646557],
 [-22.737750834794767, -47.64761945766839],
 [-22.734765962107048, -47.64227066447947],
 [-22.734753, -47.643128],
 [-22.727000570005977, -47.66539980766818],
 [-22.72764404781666, -47.676338516322446],
 [-22.727000570005977, -47.66539980766818],
 [-22.72990990843241, -47.64999847552933],
 [-22.729286, -47.65222],
 [-22.728901095296024, -47.65142593782048],
 [-22.729765757700033, -47.65208921628917],
 [-22.728565459559615, -47.64597583536602],
 [-22.727000570005977, -47.66539980

In [133]:
points_venue

[[-22.764174, -47.676697],
 [-22.735032941702027, -47.64406812173736],
 [-22.752126403483782, -47.65312260155776],
 [-22.757143, -47.68918],
 [-22.748013680889134, -47.65812158854391],
 [-22.7461718861014, -47.6712413784117],
 [-22.746669639096755, -47.661630776135695],
 [-22.766559, -47.664516],
 [-22.75939503658774, -47.6527813418554],
 [-22.743138035219392, -47.6722703300071],
 [-22.743247043556305, -47.66634691535207],
 [-22.748818, -47.673357],
 [-22.736675659078927, -47.664976739682544],
 [-22.774139404296875, -47.67181396484375],
 [-22.74079500543115, -47.66502962299508],
 [-22.738678, -47.66335],
 [-22.738161, -47.659054],
 [-22.762459, -47.68805],
 [-22.751588821411133, -47.66827392578125],
 [-22.76486612856388, -47.65860381536186],
 [-22.74298738494904, -47.657972573420594],
 [-22.767776757927688, -47.65922719778024],
 [-22.77143563790768, -47.665140197365446],
 [-22.746681021298535, -47.65898684597323],
 [-22.73661762825202, -47.66285664330137],
 [-22.737603, -47.663469],
 [

Now, we will visualize the results

In [44]:
# to plot a heatmap using folium.plugins.HeatMap, we will generate a data in the expected format
venue_matrix = np.zeros([number_x_points*number_y_points, 3])
coffe_matrix = np.zeros([number_x_points*number_y_points, 3])

# flatten the full response for easy of use
full_response_flat = list(itertools.chain(*full_response))

# now, populate the matrix with the info from "full_response" list from foursquare
i = 0
for lat in lat_range:
    for lng in lng_range:
        coffe_matrix[i] = (lat, lng, full_response_flat[i][0])
        venue_matrix[i] = (lat, lng, full_response_flat[i][1])
        i += 1

In [83]:
# and do some data threatment and formating

max_venue = venue_matrix[:,2].max()
min_venue = venue_matrix[:,2].min()
venue_matrix_normalized = (venue_matrix[:,2] - min_venue)/(max_venue - min_venue)

To plot using folium heatmap, we will create a list of points in the expected format:

In [84]:
points_venue = []
for venue in venue_matrix:
    for i in range(int((venue[2]/10))):
        points_venue.append([venue[0], venue[1]])

In [85]:
points_coffe = []
for venue in coffe_matrix:
    for i in range(int((venue[2]))):
        points_coffe.append([venue[0], venue[1]])

In [139]:
m_venues = folium.Map(location=[lat_city_center, lng_city_center], zoom_start=13)
m_venues.add_child(folium.plugins.HeatMap(points_venue, radius=8, min_opacity=0.2, blur=5, control_scale=False))
m_venues

In [153]:
m_cofee = folium.Map(location=[lat_city_center, lng_city_center], zoom_start=14)
m_cofee.add_child(folium.plugins.HeatMap(points_coffe, radius=70, min_opacity=0.2, blur=35, control_scale=False))
m_cofee

if both graphs where not able to render properly (the are some known bugs no mozilla for the heamap generation *), here are the plots as jped images

* https://github.com/python-visualization/folium/issues/812

In [91]:

Image(filename='test.png') 

FileNotFoundError: [Errno 2] No such file or directory: 'test.png'